In [ ]:
tops_base_path = 'example_images/tops'
bottoms_base_path = 'example_images/bottoms'
shoes_base_path = 'example_images/shoes'

In [ ]:
import os
import dominant_colors
from fuzzy_color import FuzzyColor
from color_matching import ColorMatching
import colorsys
import matplotlib.pyplot as plt
import numpy as np

fc = FuzzyColor()

def rgb_to_hsv(rgb):
  rgb = rgb / 255
  h, s, v = colorsys.rgb_to_hsv(*rgb)
  return h * 360, s * 100, v * 100

matches = []

for top in os.listdir(tops_base_path)[:1]:
  top_image_path = os.path.join(tops_base_path, top)
  top_color, _ = dominant_colors.get_dominant_colors(top_image_path)[0]
  top_color_hsv = rgb_to_hsv(top_color)
  top_color_desc = fc.get_color_description(*top_color_hsv)

  for bottom in os.listdir(bottoms_base_path)[:2]:
    bottom_image_path = os.path.join(bottoms_base_path, bottom)
    bottom_color, _ = dominant_colors.get_dominant_colors(bottom_image_path)[0]
    bottom_color_hsv = rgb_to_hsv(bottom_color)
    bottom_color_desc = fc.get_color_description(*bottom_color_hsv)

    for shoe in os.listdir(shoes_base_path)[:2]:
      shoe_image_path = os.path.join(shoes_base_path, shoe)
      shoe_color, _ = dominant_colors.get_dominant_colors(shoe_image_path)[0]
      shoe_color_hsv = rgb_to_hsv(shoe_color)
      shoe_color_desc = fc.get_color_description(*shoe_color_hsv)

      cm = ColorMatching([top_color_desc, bottom_color_desc, shoe_color_desc])
      valid_matches = cm.get_all_valid_matches()
      matches.append({
        "images": [top, bottom, shoe],
        "image_paths": [top_image_path, bottom_image_path, shoe_image_path],
        "dominant_colors": np.array([top_color, bottom_color, shoe_color]),
        "color_descriptions": [top_color_desc, bottom_color_desc, shoe_color_desc],
        "valid_matches": valid_matches,
      })

num = len(matches)

for i in range(num):
  fig, ax = plt.subplots(nrows=1, ncols=4)
  fig.set_size_inches(3 * 4, 5)
  clothing = ('TOP', 'BOTTOM', 'SHOES')
  y_pos = np.arange(len(clothing))
  bar_width = [1, 1, 1]

  ax[0].barh(y_pos, bar_width, align='center', color=matches[i]['dominant_colors'] / 255, height=0.99)
  ax[0].set_yticks(y_pos)
  ax[0].set_yticklabels(clothing)
  ax[0].invert_yaxis() # labels read top-to-bottom
  ax[0].set_title(str(matches[i]['images']))
  # ax[0].set_title(str(matches[i]['color_descriptions']))
  ax[0].tick_params(left=False, bottom=False, labelbottom=False)
  ax[0].text(0.5, -0.02, str(matches[i]['valid_matches']), ha='center',
            va='top', transform=ax[0].transAxes)
  ax[0].margins(y=0, x=0)

  image_path = matches[i]['image_paths'][0]
  image = plt.imread(image_path)

  image_paths = matches[i]['image_paths']
  for j, image_path in enumerate(image_paths):
      image = plt.imread(image_path)
      ax[j + 1].imshow(image)
      ax[j + 1].axis('off')
      ax[j + 1].set_title(clothing[j])
      ax[j + 1].margins(y=0, x=0)

  # print(matches[i]['images'])
  # print(matches[i]['dominant_colors'][0], matches[i]['dominant_colors'][1], matches[i]['dominant_colors'][2])
  # print(matches[i]['valid_matches'])
  # print()
